# Exercises on Transformations

## Exercise 4.1
Implement a `translate_by(...)` function that takes a translation vector as an input and returns a translation function as output.

Let's define the function and apply it to the teapot to validate it works

In [2]:
from teapot import load_triangles
from draw_model import draw_model
from vectors import add

def polygon_map(transformation, polygons):
    return [
        [transformation(vertex) for vertex in polygon]
        for polygon in polygons
    ]

def translate_by(vector):
    def new_function(v):
        return add(vector, v)
    return new_function

draw_model(polygon_map(translate_by((-1, 0, 0)), load_triangles()))

SystemExit: 0

| NOTE: |
| :---- |
| To help simplify running the cells, I have created a library `transformation_support.py` that contains the building blocks such as `polygon_map(...)` etc. so that we don't have to create them in each and every exercise. |

## Exercise 4.2

Render the teapot translated by 20 units in the negative z direction. What does the resulting image look like?

It should look as if we're looking at the teapot way above 3D model. We should be able to see the top of the teapot, and it should also look smaller, as it is way further.

In [1]:
from teapot import load_triangles
from draw_model import draw_model
from transformations_support import *

draw_model(polygon_map(translate_by((0, 0, -20)), load_triangles()))

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 0

Note that one of our predictions is right (it's smaller), but not the other (the shape it is still centered).
The shape is further from the viewpoint `(0, 0, -5)`.

Let's see what happens when we translate from different vectors:

In [2]:
from teapot import load_triangles
from draw_model import draw_model
from transformations_support import *

draw_model(polygon_map(translate_by((-2, 0, 0)), load_triangles()))

SystemExit: 0

In [3]:
from teapot import load_triangles
from draw_model import draw_model
from transformations_support import *

draw_model(polygon_map(translate_by((0, -2, 0)), load_triangles()))

SystemExit: 0

So, what we though would happen for `(0, 0, -20)` is happening for `(0, -2, 0)`. The translation by `(-2, 0, 0)` is consistent with our expectations (move to the left) and `(0, 0, -20)` is more consisten with what we would expect for movement on the y axis.

## Exercise 4.3

What happens to the teapot when you scale every vector by a scalar between 0 and 1? What happens when you scale it by a factor of -1?

When you scale every vector by a scalar between 0 and 1 you will make the teapot smaller, but maintaining the ratio. Let's confirm:

In [2]:
from teapot import load_triangles
from draw_model import draw_model
from transformations_support import *

draw_model(polygon_map(scale_by(0.1), load_triangles()))

SystemExit: 0

If we scale every vector of the teapot by -1, we will *negate* every vector of the 3D model. As a result up will be down and left will be right (and vice versa).

In [4]:
from teapot import load_triangles
from draw_model import draw_model
from transformations_support import *

draw_model(polygon_map(scale_by(-1), load_triangles()))

SystemExit: 0

It must be noted that actually, scaling by -1 turns every vector inside-out, so each normal vector will now point into the teapot rather than outward, and therefore will create problem while rendering.

Let's rotate the scaled teapot to see that:

In [3]:
from teapot import load_triangles
from draw_model_custom_impl import draw_model_anim
from transformations_support import *

draw_model_anim(polygon_map(scale_by(-1), load_triangles()))


SystemExit: 0

In [ ]:
See how it does not render correctly, specially the spout.

## Exercise 4.4

First apply a `translate1left(...)` to the teapot and then `scale2`. How is the result different when you compose the functions in different order?

We know that if we first scale and then translate, a double size teapot will be found translated to the left. However, if we first translate and scale, the teapot will be stretched:

In [3]:
from teapot import load_triangles
from draw_model_custom_impl import draw_model_anim
from transformations_support import *

draw_model_anim(polygon_map(compose(translate_by((-1, 0, 0)), scale_by(2)), load_triangles()), display_axes=True)

SystemExit: 0

In [4]:
from teapot import load_triangles
from draw_model_custom_impl import draw_model_anim
from transformations_support import *

draw_model_anim(polygon_map(compose(scale_by(2), translate_by((-1, 0, 0))), load_triangles()), display_axes=True)

SystemExit: 0

So we were wrong again: if we first translate and then scale, the 3D model is translated further to the left. This is because when the scaling factor of 2 is applied after the translation, the distance of the translation doubles as well.

![Transformation Order is important](../images/transformation_order.png)

## Exercise 4.5

What is the effect of the transformation `compose(scale_by(0.4), scale_by(1.5))`?

The effect will be the scaling the 3D shape by $ 0.4 \times 1.5 = 0.6 $

In [1]:
from teapot import load_triangles
from draw_model_custom_impl import draw_model_anim
from transformations_support import *

draw_model_anim(polygon_map(compose(scale_by(0.4), scale_by(1.5)), load_triangles()))

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 0

## Exercise 4.6

Modify the `compose(...)` function so that it can take a variable number of arguments that will be applied in given order.

Let's see what is the anatomy of the `compose(f1, f2)` function and think about how it can be generalized:

```python
def compose(f1, f2):
    def new_function(input):
        return f1(f2(input))
    return new_function
```

In [3]:
def compose_multiple(*fArgs):
    def new_function(input):
        resulting_f = None
        for f in fArgs:
            resulting_f = f(input)
        return resulting_f
    return new_function

from teapot import load_triangles
from draw_model_custom_impl import draw_model_anim
from transformations_support import *

draw_model_anim(polygon_map(compose_multiple(scale_by(0.4), scale_by(1.5), scale_by(2)), load_triangles()))

SystemExit: 0

This can be further improved, by applying it in reversed order, so that it is applied in the same order as `compose(f_last, f_first)`. Also, if no function given the *identity* transformation is applied (no transformation).

In [ ]:
def compose(*fArgs):
    def new_function(input):
        state = input
        for f in reversed(args):
            state = f(state) 
            return state     
    return new_function

In [1]:
from teapot import load_triangles
from draw_model_custom_impl import draw_model_anim
from transformations_support import *

draw_model_anim(polygon_map(compose(scale_by(0.4), scale_by(1.5), scale_by(2), translate_by((-1, 0, 0))), load_triangles()))

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 0

Note that the `compose(...)` function is not constrained to 3D transformations, it can be applied to all type of Python functions that receive an input and transforms it:

In [3]:
def prepend(string):
    def new_function(input):
        return string + input
    return new_function


f = compose(prepend('P'), prepend('Y'), prepend('T'))
print(f('HON'))


PYTHON


## Exercise 4.7

Write a `curry2(f)` that takes a Python function `f(x, y)` with two arguments and returns a curried version.

That is, `g = curry2(f)` the result of `f(x, y)` and `g(x)(y)` should return the same result.

Currying is the process of transforming a function that takes multiple arguments in a function that takes a single argument and returns another function which accepts further arguments, one by one, that the original function would receive.

In our case, we have a function `f(x, y)` that receives two arguments. Therefore, we'll use *currying* to transform it into `g(x)(y)`.

Let's do it in Python:

In [6]:
# no currying
scale_by_2 = scale_by(2)
print(scale_by_2((1, 2, 3)))


def curry2(f):
    def g(x):
        def new_function(y):
            return f(x, y)
        return new_function
    return g

scale_by = curry2(scale)

print(scale_by(2)((1, 2, 3)))

(2, 4, 6)
(2, 4, 6)


## Exercise 4.8

Without running it, what is the result of applying the transformation `compose(rotate_z_by(pi/2), rotate_x_by(pi/2))? What if you switch the order of the composition?

The result will be the teapot rotated by the x-axis 90° towards us, so that we see the top part of the teapot, and then rotated 90° about the z-axis, so that the spout is looking down.

Let's confirm:

In [1]:
from teapot import load_triangles
from draw_model_custom_impl import draw_model_anim
from transformations_support import *
from math import pi

draw_model_anim(polygon_map(rotate_z_by(pi / 2.), load_triangles()))

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 0

In [1]:
from teapot import load_triangles
from draw_model_custom_impl import draw_model_anim
from transformations_support import *
from math import pi

draw_model_anim(polygon_map(rotate_x_by(pi / 2.), load_triangles()))

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 0

In [6]:
from teapot import load_triangles
from draw_model_custom_impl import draw_model_anim
from transformations_support import *
from math import pi

draw_model_anim(polygon_map(compose(rotate_z_by(pi / 2.), rotate_x_by(pi / 2.)), load_triangles()))

SystemExit: 0

If we apply the reverse composition, the spout will be in our direction:

In [5]:
from teapot import load_triangles
from draw_model_custom_impl import draw_model_anim
from transformations_support import *
from math import pi

draw_model_anim(polygon_map(compose(rotate_x_by(pi / 2.), rotate_z_by(pi / 2.)), load_triangles()))

SystemExit: 0

## Exercise 4.9

Write a function `stretch_x(scalar, vector)` that scales the target vector by the given factor, but only in the x direction.

Also, write a curried version of `stretch_x_by(...)` so that `stretch_x_by(scalar)(vector)` returns the same result.

I'm already used to *currying* as seen in the previous example, without currying it would be:

```python
def stretch_x(scalar, vector):
    ...
```



In [2]:
from teapot import load_triangles
from draw_model_custom_impl import draw_model_anim
from transformations_support import *

# no currying

def stretch_x(scalar, vector):
    x, y, z = vector
    return (scalar * x, y, z)

transformed_triangles = [
    [stretch_x(1.5, v) for v in triangle]
    for triangle in load_triangles()
]

draw_model_anim(transformed_triangles)

SystemExit: 0

In [4]:

def stretch_x(scalar):
    def new_function(v):
        x, y, z = v
        return (scalar * x, y, z)
    return new_function

draw_model_anim(polygon_map(stretch_x(0.5), load_triangles()))


SystemExit: 0